In [3]:
import numpy as np
import pandas as pd
#from haversine import haversine, Unit
import geopy.distance

In [4]:
large_df = pd.read_csv('../DAOU/test_large_out_v2.csv', index_col=0, encoding='euc-kr')
large_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   위도                      5774 non-null   float64
 6   경도                      5774 non-null   float64
 7   별점                      310 non-null    float64
 8   방문자 리뷰수                 685 non-null    object 
 9   블로그 리뷰수                 513 non-null    object 
 10  지하철역과의 거리               774 non-null    object 
 11  도보시간                    94 non-null     object 
 12  한실수                     5434 non-null   float64
 13  양실수                     5347 non-null   float64
 14  욕실수                     4813 non-null   

In [5]:
subway = pd.read_csv('../DABA/서울시 역사마스터 정보.csv', index_col=0, encoding='euc-kr')
subway

,역사명,호선,위도,경도
역사_ID,,,,
9996,미사,5호선,127.193877,37.560927
9995,강일,5호선,127.175930,37.557490
4929,김포공항,김포골드라인,126.801868,37.562360
4928,고촌,김포골드라인,126.770345,37.601243
4927,풍무,김포골드라인,126.732387,37.612488
...,...,...,...,...
154,종로5가,1호선,127.001849,37.570926
153,종로3가,1호선,126.991847,37.570406
152,종각,1호선,126.982923,37.570161


In [9]:
large_df.iloc[0]['사업장명']

print(np.nan)

nan


In [10]:
# .. 가까운 지하철 거리 및 도보 시간 구하기
# 가까운 지하철 구하기

large_dist = []

for i in range(large_df.shape[0]):
#for i in range(1):
    dummy = []

    if np.isnan(large_df.iloc[i]['위도']) :

        dummy.append(np.nan)
        find_min_j = np.nan
        find_stn_nam = np.nan
        find_stn_lat = np.nan
        find_stn_lon = np.nan
        nearest_stn = np.nan
        time_to_near = np.nan

    else:

        for j in range(subway.shape[0]):

            hotel_latlon = (large_df.iloc[i]['위도'], large_df.iloc[i]['경도'])
            subway_latlon = (subway.iloc[j]['경도'], subway.iloc[j]['위도'])    # 원본데이터에서 위경도 컬럼명 바뀌어 있음

            dist = geopy.distance.geodesic(hotel_latlon, subway_latlon).m
            dummy.append(dist)

        find_min_j = np.argmin(dummy)
        dist_to_stn = np.nanmin(dummy)
        time_to_stn = np.nanmin(dummy)/66.7

        find_stn_nam = subway.iloc[find_min_j]['역사명']
        find_stn_lat = subway.iloc[find_min_j]['위도']
        find_stn_lon = subway.iloc[find_min_j]['경도']

        
            
        #print(f"호텔:{large_df.iloc[i]['사업장명']} {large_df.iloc[i]['지번주소']} {min(dummy)}m")
    large_dist.append([large_df.iloc[i]['사업장명'],
                        large_df.iloc[i]['지번주소'],
                        large_df.iloc[i]['위도'],
                        large_df.iloc[i]['경도'],
                        find_stn_nam,
                        find_stn_lat,
                        find_stn_lon,
                        dist_to_stn,
                        time_to_stn]) # 시속 4km --> 분속 66.7m



In [11]:
find_large_dist = pd.DataFrame(large_dist, columns=['사업장명','지번주소','호텔 위도', '호텔 경도','역이름', '역 위도','역 경도', '거리','도보시간(분)'])

#for i in range(10):
#    print(f"호텔:{large_df.iloc[i]['사업장명']}, {df.iloc[i]['거리']}")
find_large_dist.info()
find_large_dist

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6664 entries, 0 to 6663
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     6664 non-null   object 
 1   지번주소     6653 non-null   object 
 2   호텔 위도    5774 non-null   float64
 3   호텔 경도    5774 non-null   float64
 4   역이름      5774 non-null   object 
 5   역 위도     5774 non-null   float64
 6   역 경도     5774 non-null   float64
 7   거리       6664 non-null   float64
 8   도보시간(분)  6664 non-null   float64
dtypes: float64(6), object(3)
memory usage: 468.7+ KB


,사업장명,지번주소,호텔 위도,호텔 경도,역이름,역 위도,역 경도,거리,도보시간(분)
0,어반스테이 명동,서울특별시 중구 인현동1가 158,37.563358,126.993360,충무로,126.994072,37.561430,223.010258,3.343482
1,오요5(oYo5),서울특별시 중구 남산동2가 35-2,37.558934,126.984838,명동,126.986325,37.560989,263.215774,3.946263
2,더스테이클래식호텔,서울특별시 중구 남대문로3가 94,37.561480,126.978525,회현(남대문시장),126.978246,37.558514,330.151983,4.949805
3,오요5롯지A(oYo5LodgeA),서울특별시 중구 남산동2가 33-3 1~2층,37.559108,126.984981,명동,126.986325,37.560989,240.223310,3.601549
4,오요5롯지B(oYo5LodgeB),"서울특별시 중구 남산동2가 33 1,2층",37.559222,126.985041,명동,126.986325,37.560989,226.602301,3.397336
...,...,...,...,...,...,...,...,...,...
6659,이녹(李綠),서울특별시 종로구 종로5가 66-1,37.571482,127.002656,종로5가,127.001849,37.570926,94.265899,1.413282
6660,설레임모텔,서울특별시 강서구 화곡동 24-77,37.546468,126.846581,화곡,126.840461,37.541513,771.321547,11.564041
6661,신라장,서울특별시 동작구 신대방동 687-8,37.488098,126.908444,신대방,126.913149,37.487462,422.041104,6.327453
6662,웨이브,서울특별시 성북구 정릉동 408-11,37.606978,127.010455,정릉,127.013396,37.603133,499.540547,7.489364


In [12]:
large_df['가까운 지하철역'] = find_large_dist['역이름']
large_df['지하철역과의 거리'] = find_large_dist['거리']
large_df['도보시간'] = find_large_dist['도보시간(분)']

In [13]:
large_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   위도                      5774 non-null   float64
 6   경도                      5774 non-null   float64
 7   별점                      310 non-null    float64
 8   방문자 리뷰수                 685 non-null    object 
 9   블로그 리뷰수                 513 non-null    object 
 10  지하철역과의 거리               6664 non-null   float64
 11  도보시간                    6664 non-null   float64
 12  한실수                     5434 non-null   float64
 13  양실수                     5347 non-null   float64
 14  욕실수                     4813 non-null   

In [16]:
colum_names2 = ['사업장명','업소유형','업태구분명','지번주소','도로명주소','가까운 지하철역', '지하철역과의 거리', '도보시간',
                '위도','경도', '별점','방문자 리뷰수', '블로그 리뷰수', '한실수','양실수','욕실수','구비시설',
                '네이버 이런점이 좋아요 총합', '네이버 이런점이 좋아요 {항목:좋아요수}', '데이터갱신일자',
                '인허가일자','인허가취소일자','영업상태코드','폐업일자','휴업시작일자','휴업종료일자','재개업일자']  # 26

large_df = large_df[colum_names2]
large_df

KeyError: "['가까운 지하철역'] not in index"

In [ ]:
output_path = "../DAOU/test_small_out_v3.csv"
large_df.to_csv(output_path, encoding='euc-kr')